### Hi and welcome to this guide where you will see how easy classify Lego minifigures from  [LEGO Minifigures Classification](https://www.kaggle.com/ihelon/lego-minifigures-classification) dataset.

The task is that we send a picture to the neural network and the neural network returns the class of minifigures. So, if we send a picture of Ron Weasley  minifigure, then the neural network will return "class 19" and the name "RON WEASLEY".

Here we will use pretrained neural network [DenseNet121](https://keras.io/api/applications/densenet/) from Keras, because it will done well with this task and I have not reason to train my own neural network if we can use existing models. We will also use [Dence](https://medium.com/datathings/dense-layers-explained-in-a-simple-way-62fe1db0ed75) and [Dropout](https://medium.com/analytics-vidhya/a-simple-introduction-to-dropout-regularization-with-code-5279489dda1e) layers, so if you don't know what is it just click and read. 

*Thank you for reading my works ♥, you can also see my other guide:*

-- [Guide for beginners. Breast cancer classification.](https://www.kaggle.com/izabellaleroy/guide-for-beginners-breast-cancer-classification)

*Please, also like  the [LEGO Minifigures Classification](https://www.kaggle.com/ihelon/lego-minifigures-classification) dataset from [ihelon](https://www.kaggle.com/ihelon), because this is  very painstaking work to create the dataset.*

**Let's go!**

Just import all we will need.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import tensorflow as tf 
import pandas as pd
import numpy as np
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

First of all let's look what we have in dataset.

There are pictures of Lego minifigures from different universes (Star Wars, Harry Potter, etc.) in our dataset. Each picture is 512x512 picsels. Figures are photographed from different angles and in different locations.

**The code below is just to see a few images.**

In [ ]:
images = [[],[],[]]
index = ['001', '002', '003']

for i in range(3):
    images[i].append(mpimg.imread('../input/lego-minifigures-classification/star-wars/0001/' + index[i] + '.jpg'))
    images[i].append(mpimg.imread('../input/lego-minifigures-classification/marvel/0001/' + index[i] + '.jpg'))
    images[i].append(mpimg.imread('../input/lego-minifigures-classification/harry-potter/0001/' + index[i] + '.jpg'))

fig, axs = plt.subplots(3,3, figsize=(18,18))

for i in range(3):
    for j in range(3):
        axs[i, j].imshow(images[i][j])

Download data

In [ ]:
data = pd.read_csv('../input/lego-minifigures-classification/index.csv')
data

Above we see that we already have labels for train and validation, so lets just split it.

In [ ]:
train_set = data[data["train-valid"] == 'train']
validation_set = data[data["train-valid"] == 'valid']

**Let's already create the model.**

DenceNet ([Densely Connected Convolutional Networks](https://arxiv.org/abs/1608.06993)) was created by Gao Huang, Zhuang Liu, Laurens van der Maaten, Kilian Q. Weinberger. This model has been trained on a very large amount of data and is able to distinguish many different objects.

As I already said, we will take a pre-trained DenceNet model, but change the last layer in it and also add our own output layer, so this model will classify what we need, in our case, Lego minifigures. 

See the picture below:

<img src="https://i.imgur.com/ed9PLCI.png" width="500">

In [ ]:
# take pretrained DenseNet
base_model = tf.keras.applications.DenseNet121()
# Create new Dropout layer and sent there penultimate output from DenseNet 
my_layer = Dropout(0.5)(base_model.layers[-2].output)
# Count the nuber of unique classes in dataset
number_of_classes = len(data['class_id'].unique())
# Create new Dense layer and sent there output from Dropout layer
# Note that in this Dense layer is "number_of_classes" neuron because we have "number_of_classes" classes 
my_outputs = Dense(number_of_classes, activation="softmax")(my_layer)
model = Model(base_model.input, my_outputs)

*Note:* last Dense layer has "number_of_classes" neuron because we have "number_of_classes" classes, so the output from our model will be a vector with "number_of_classes" float numbers and the answer will be the index of the max element.

Now we must compile our model.

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

The neural network accepts an image as input in the form of pixel values, so the code below simply transforms the images from the training and validation set into arrays of numbers and saves them. 

In [ ]:

X_train = np.zeros((train_set.shape[0], 512, 512, 3))

for i in range(train_set.shape[0]):
    image = cv2.imread('../input/lego-minifigures-classification/' + train_set["path"].values[i])
    image = cv2.resize(image, dsize=(512,512)) # resize in case if image was not 512x512 pixels
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    X_train[i] = image/255

Y_train = np.array(train_set["class_id"])-1

X_valid = np.zeros((validation_set.shape[0], 512, 512, 3))

for i in range(validation_set.shape[0]):
    image = cv2.imread('../input/lego-minifigures-classification/' + validation_set["path"].values[i])
    image = cv2.resize(image, dsize=(512,512))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    X_valid[i] = image/255

Y_valid = np.array(validation_set["class_id"])-1

This is lables in train set.

*Note:* Labels must be from 0 to n, but in our dataset labels from 1 to n, so above wen we save Y_valid and Y_train we subtract 1. 
I draw your attention to this, because in the future when we receive the answer class we will have to add 1 for the right answer.

In [ ]:
Y_train

Before training the model, *let's create the checkpoint* for the model. It's mean that we will save only the best model. We do this because there may be a situation when, for example, we train the model for 50 epochs, but the best result was at 43 epochs, so if we do not save it, we will lose the best result and get only the final result from 50 epochs.

The model will be saved at Kaggle's work directory "./kaggle/working". We will name our best model just "model.h5". We will choose the best model by value accuracy score (monitor="val_accuracy").

In [ ]:
checkpoint = ModelCheckpoint(filepath='model.h5', monitor="val_accuracy", save_best_only=True, verbose=1)

Now let's train our model!

*Note: the function fit() has a parameter "callbacks" where we will pass our checkpoint.* 

In [ ]:
model.fit(
    X_train, 
    Y_train, 
    epochs=50, 
    validation_data=(X_valid, Y_valid), 
    shuffle=True, 
    batch_size=4, 
    callbacks=checkpoint
)

Great! Now we have a trained model. But the variable "model" contains not the best weights. But that weights for the model which trained for 50 epoch. So let's just download our saved, the best model from "./kaggle/working/model.h5".

In [ ]:
model = load_model("model.h5")

That's all for now, below you can download and test your own picture or some pictures from the dataset!
Write comments or ask questions if something is not clear, also click like 😍 if you liked it and read my other works!

In [ ]:
image = cv2.imread('../input/lego-minifigures-classification/harry-potter/0002/009.jpg') # read the image 
image = cv2.resize(image, dsize=(512,512)) # resize in case if image was not 512x512 pixels
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)/255 # transform picture from BRG to the RGB format

plt.imshow(image) # print image 

image = np.reshape(image, (1, 512, 512, 3)) # resize to the needed for model shape - 1 picture 512 height 512 width 3 chanel(RGB)

ans = model.predict(image).argmax() # find index of max element
ans = ans+1 # don't forget to add 1 :) 
metadata = pd.read_csv('../input/lego-minifigures-classification/metadata.csv') # download meta data, there are store real 
                                                                                #names of minifigures

minifigure = metadata["minifigure_name"][metadata["class_id"] == ans].iloc[0] # find the name that matches the predicted class
print(f"Class:\t{ans}\tMinifigure:\t{minifigure}")